In [10]:
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
import h5py
import numpy as np
import scipy.linalg as la
from scipy.optimize import minimize
import pandas as pd
import networkx as nx
import cirq
from cqaoa.ansatz import CylicQAOAAnsatz
from cqaoa.maxcut import edge_operator, maxcut_hamiltonian, bitstring_energy

In [11]:
n = 8
lr = n // 2
p = 4
qs = cirq.LineQubit.range(n)
qubit_graph = nx.Graph()
lefts = [i for i in range(lr)]
rights = [i for i in range(lr,n)]
for left in lefts:
    for right in rights:
        qubit_graph.add_edge(qs[left], qs[right], weight=1.0)
rng_inst = np.random.default_rng(9090)
h = rng_inst.normal(0., 0.35, size=n)
for i in range(n):
    qubit_graph.add_edge(qs[i],qs[i],weight = h[i])

hamiltonian = maxcut_hamiltonian(qubit_graph)
alpha = np.zeros(p)
print(alpha.shape)
ref = np.array([False] * n, dtype=bool)
ansatz = CylicQAOAAnsatz(qubit_graph, hamiltonian, weighted=True, alpha=alpha, reference=ref)

(4,)


In [12]:
def objective_callback(ansatz: CylicQAOAAnsatz, params: np.ndarray) -> float:
    """Get the energy expectation for certain parameters."""

    assert params.size % 2 == 0
    p = params.size // 2
    gammas = params[:p]
    betas = params[p:]
    return ansatz.energy(gammas, betas)


def optimize_ansatz(ansatz: CylicQAOAAnsatz, gamma0: np.ndarray, beta0: np.ndarray) -> Tuple[np.ndarray, np.ndarray, float]:
    """Minimize the energy, returning the optimal gamma and beta parameters."""

    assert gamma0.size == beta0.size

    p = gamma0.size
    params0 = np.concat([gamma0, beta0])
    opt_result = minimize(
        lambda params: objective_callback(ansatz, params),
        params0, method="Powell", options={"maxiter": 1_000_000}
    )
    assert opt_result.success, f"Optimizer failed: {opt_result.message}"
    optimized_energy = objective_callback(ansatz, opt_result.x)
    gamma_opt = opt_result.x[:p]
    beta_opt = opt_result.x[p:]
    return (gamma_opt, beta_opt, optimized_energy)

In [13]:
def cirq_samples_to_sorted_ndarray(samples: pd.DataFrame) -> np.ndarray:
    """Cirq returns a dataframe of samples with column names like q(12).
    We want to sort these clumns by their integer values and then convert the DataFrame
    to an np array of booleans."""

    nsamples = len(samples)
    ncols = len(samples.columns)

    col_name_index_pairs: List[Tuple[str, int]] = []
    for col_name in samples.columns:
        digits = ''.join(c for c in col_name if c.isdigit())
        if len(digits) == 0:
            raise ValueError(f"Column name {col_name} has no digits.")
        col_name_index_pairs.append((col_name, int(digits)))
    # Sort the column name, index pairs by the index.
    sorted_indices = sorted(col_name_index_pairs, key=lambda t: t[1])
    sorted_column_names = [t[0] for t in sorted_indices]
    # Put columnns into the array in order.
    samples_np = np.zeros((nsamples, ncols), dtype=bool)
    for i, col in enumerate(sorted_column_names):
        bool_column = samples[col].to_numpy()
        samples_np[:, i] = bool_column.copy()
    return samples_np

In [14]:
# Get a reference state by sampling lots of bitstrings
# from an optimized Ansatz with alpha = 0.

alpha = np.zeros(p)
print(alpha.shape)
ref = np.array([False] * n, dtype=bool)
ansatz = CylicQAOAAnsatz(qubit_graph, hamiltonian, weighted=True, alpha=alpha, reference=ref)

gamma0 = np.random.rand(p)
beta0 = np.random.rand(p)
gamma_opt, beta_opt, optimize_energy = optimize_ansatz(ansatz, gamma0, beta0)

shots = 10_000
bitstrings_df = ansatz.sample_bitstrings(gamma_opt, beta_opt, shots)
bitstrings_array = cirq_samples_to_sorted_ndarray(bitstrings_df).T
bitstrings = [bitstrings_array[:, i] for i in range(bitstrings_array.shape[1])]
energies = [bitstring_energy(bs, hamiltonian) for bs in bitstrings]
i_min = np.argmin(energies)
energy_min = energies[i_min]
ref = bitstrings[i_min]
print(f"Got energy {energy_min} with bitstring\n{ref}")

(4,)
Got energy -20.0 with bitstring
[ True  True  True  True False False False False]


In [15]:
def entanglement_stats(psi: np.ndarray, nqubits: int) -> Dict[str, float]:
    """Get mean, std, min, and max von Neumann entropy across all bipartitions
    on the line of qubits. For example, if there are 4 qubits, then there are three possible
    bipartitions: between qubits 0 and 1, between 1 and 2, and between 2 and 3."""

    rho = np.outer(psi, psi.conj())
    rho_tensor = rho.reshape((2,) * (2 * nqubits))
    entropies: List[float] = []
    for i in range(1, nqubits - 1):
        rho_reduced = cirq.partial_trace(rho_tensor, range(i))
        rho_reduced_matrix = rho_reduced.reshape((2 ** i, 2 ** i))
        entropy = cirq.von_neumann_entropy(rho_reduced_matrix)
        entropies.append(entropy)
    results = {}
    results["min"] = np.min(entropies)
    results["max"] = np.max(entropies)
    results["mean"] = np.mean(entropies)
    results["std"] = np.std(entropies)
    return results

In [16]:
alpha = 0.2 * np.ones(p)
ref = np.array([False] * n, dtype=bool)
ansatz = CylicQAOAAnsatz(qubit_graph, hamiltonian, weighted=True, alpha=alpha, reference=ref)
gamma0 = np.random.rand(p)
beta0 = np.random.rand(p)
gamma_opt, beta_opt, energy_opt = optimize_ansatz(ansatz, gamma0, beta0)
print(f"Optimized energy: {energy_opt}")

Optimized energy: -16.775839017063845


In [17]:
ckt = ansatz.circuit(gamma_opt, beta_opt)
sim = cirq.Simulator()
sim_result = sim.simulate(ckt)
psi = sim_result.final_state_vector
entropy_results = entanglement_stats(psi, n)
for k, v in entropy_results.items():
    print(k, v)

min 0.9726496
max 1.7495462
mean 1.4813786
std 0.25998184


In [18]:
@dataclass
class EntropyResult:
    """Result of evaluating entanglement entropy."""

    alpha: float
    gamma: np.ndarray
    beta: np.ndarray
    psi: np.ndarray
    energy: float
    min_entropy: float
    max_entropy: float
    mean_entropy: float
    std_entropy: float


def entropy_result_for_alpha(p: int, alpha: float, ref: np.ndarray) -> EntropyResult:
    """Optimize the Ansatz for a fixed alpha value (alpha_0 = ... = alpha_p),
    then get statistics of the entanglement for the final state."""

    alphas = alpha * np.ones(p)
    ref = np.array([False] * n, dtype=bool)
    ansatz = CylicQAOAAnsatz(qubit_graph, hamiltonian, weighted=True, alpha=alphas, reference=ref)
    gamma0 = np.random.rand(p)
    beta0 = np.random.rand(p)
    gamma_opt, beta_opt, energy_opt = optimize_ansatz(ansatz, gamma0, beta0)
    ckt = ansatz.circuit(gamma_opt, beta_opt)
    sim = cirq.Simulator()
    sim_result = sim.simulate(ckt)
    psi = sim_result.final_state_vector
    entropy_dict = entanglement_stats(psi, n)
    entropy_result = EntropyResult(
        alpha, gamma_opt, beta_opt, psi, energy_opt,
        entropy_dict["min"], entropy_dict["max"], entropy_dict["mean"], entropy_dict["std"]
    )
    return entropy_result

In [19]:
class EntropyResultCollection:

    def __init__(
        self, n: int = 0,
        alphas: List[float] = [],
        gammas: List[np.ndarray] = [],
        betas: List[np.ndarray] = [],
        psis: List[np.ndarray] = [],
        energies: List[float] = [],
        min_entropies: List[float] = [],
        max_entropies: List[float] = [],
        mean_entropies: List[float] = [],
        std_entropies: List[float] = []
    ):
        self._n = n
        self._alphas = alphas
        self._gammas = gammas
        self._betas = betas
        self._psis = psis
        self._energies = energies
        self._min_entropies = min_entropies
        self._max_entropies = max_entropies
        self._mean_entropies = mean_entropies
        self._std_entropies = std_entropies
    
    def append(self, result: EntropyResult):
        """Append an instance of EntropyResult to the collection."""

        self._n += 1
        self._alphas.append(result.alpha)
        self._gammas.append(result.gamma)
        self._betas.append(result.beta)
        self._psis.append(result.psi)
        self._energies.append(result.energy)
        self._min_entropies.append(result.min_entropy)
        self._max_entropies.append(result.max_entropy)
        self._mean_entropies.append(result.mean_entropy)
        self._std_entropies.append(result.std_entropy)
    
    def entropy_dataframe(self) -> pd.DataFrame:
        """Conver the entopy statistics and energies into a DataFrame, with the value of alpha serving as the index."""

        records = []
        for i in range(self._n):
            records.append(
                (self._alphas[i], self._energies[i],
                self._min_entropies[i], self._max_entropies[i], 
                self._mean_entropies[i], self._std_entropies[i])
            )
        df = pd.DataFrame.from_records(
            records, 
            columns=["alpha", "energy", "min_entropy", "max_entropy", "mean_entropy", "std_entropy"]
        )
        df.set_index("alpha", inplace=True)
        return df
    
    def to_hdf5(self, fname: str):
        """Convert the collection into an HDF5 file."""

        f = h5py.File(fname, "w")
        f.create_dataset("n", data=self._n)
        f.create_dataset("alphas", data=np.vstack(self._alphas).T)
        f.create_dataset("betas", data=np.vstack(self._betas).T)
        f.create_dataset("psis", data=np.vstack(self._psis).T)
        f.create_dataset("energies", data=np.array(self._energies))
        f.create_dataset("min_entropy", data=np.array(self._min_entropies))
        f.create_dataset("max_entropy", data=np.array(self._max_entropies))
        f.create_dataset("mean_entropy", data=np.array(self._mean_entropies))
        f.create_dataset("std_entropy", data=np.array(self._std_entropies))
        f.close()

In [21]:
alphas = np.linspace(0., 1., num=3)
entropy_result_collection = EntropyResultCollection()
for alpha in alphas:
    entropy_result = entropy_result_for_alpha(p, alpha, ref)
    entropy_result_collection.append(entropy_result)

In [22]:
df = entropy_result_collection.entropy_dataframe()
print(df.head())

          energy  min_entropy  max_entropy  mean_entropy  std_entropy
alpha                                                                
0.0   -17.315802     0.971548     1.589768      1.364430     0.203053
0.5   -18.458953     0.999229     1.392565      1.270965     0.131488
1.0   -18.023741     0.975651     1.704475      1.429140     0.239761


In [23]:
entropy_result_collection.to_hdf5("../data/entropy_results.hdf5")